In [4]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from imblearn.combine import SMOTEENN

import pickle
import pandas as pd
import numpy as np
from flask import Flask, render_template, jsonify, request
import warnings 


In [5]:
with open('HeartDisease.pkl', 'rb') as HD:
    HD_model = pickle.load(HD)
with open('diabetic.pkl', 'rb') as DI:
    DI_model = pickle.load(DI)
with open('KindneyDisease.pkl', 'rb') as KD:
    KD_model = pickle.load(KD)
with open('Stroke.pkl', 'rb') as ST:
    ST_model = pickle.load(ST)
with open('Asthma.pkl', 'rb') as AS:
    AS_model = pickle.load(AS)


In [6]:
# ##### 심장병 예측 ######## 
def HD_ML_predict(HD_model, HD_features):
    HD_pred = HD_model.predict(HD_features)
    return HD_pred

# ##### 당뇨병 예측 ######## 
def DI_ML_predict(DI_model, DI_features):
    DI_pred = DI_model.predict(DI_features)
    return DI_pred

# # ##### 신장병 예측 ########
def KD_ML_predict(KD_model, KD_features):
    KD_pred = KD_model.predict(KD_features)
    return KD_pred

# # ##### 뇌졸증 예측 ########
def ST_ML_predict(ST_model, ST_features):
    ST_pred = ST_model.predict(ST_features)
    return ST_pred

# # ##### 천식 예측 ########
def AS_ML_predict(AS_model, AS_features):
    AS_pred = AS_model.predict(AS_features)
    return AS_pred

app = Flask(__name__)

@app.route('/', methods=['GET','POST'])
def main():
    ### Home Page ###
    return render_template('homepage.html')

@app.route('/test', methods=['GET','POST'])
def test():
    ### Test page ###
    return render_template('test_page.html')

@app.route('/result_page', methods=['POST'])
def result():
    ### Result Page ###
    ### 각 설문지 값 ###
    taller = request.form['taller']
    weight = request.form['weight']
    age = request.form['age']
    sex = request.form['sex']
    sleeptime = request.form['sleeptime']
    smoking = request.form['smoking']
    alchole = request.form['alchole']
    physicalactivity = request.form['physicalactivity']
    diffwalking = request.form['diffwalking']
    heartdisease = request.form['heartdisease']
    diebtic = request.form['diebtic']
    stroke = request.form['stroke']
    kidneydisease = request.form['kidneydisease']
    Asthma = request.form['Asthma']

    dif_heartdisease = request.form['heartdisease']
    dif_diebtic = request.form['diebtic']
    dif_stroke = request.form['stroke']
    dif_kidneydisease = request.form['kidneydisease']
    dif_Asthma = request.form['Asthma']


    ### 타입 변경 ###
    taller = int(taller)
    weight = int(weight)
    age = int(age)
    sex = int(sex)
    sleeptime = int(sleeptime)
    smoking = int(smoking)
    alchole = int(alchole)
    physicalactivity = int(physicalactivity)
    diffwalking = int(diffwalking)
    heartdisease = int(heartdisease)
    Asthma = int(Asthma)
    kidneydisease = int(kidneydisease)
    stroke = int(stroke)
    diebtic = int(diebtic)

    ### 예외 값 설정 ###
    dif_heartdisease = int(dif_heartdisease)
    dif_diebtic = int(dif_diebtic)
    dif_stroke = int(dif_stroke)
    dif_kidneydisease = int(dif_kidneydisease)
    dif_Asthma = int(dif_Asthma)

    ### BMI 측정 ###
    BMI = weight / (taller * taller / 10000)
    BMI = round(BMI,2)
    
    ### BMI 값 범위 지정하기 ###
    if 0 < BMI < 18.5:
        BMI = 1
    elif 18.5 < BMI < 24.99:
        BMI = 2
    elif 25.00 < BMI < 29.99:
        BMI = 3
    elif 30.00 < BMI < 34.99:
        BMI = 4
    elif 35.00 < BMI < 39.99:
        BMI = 5
    else:
        BMI = 6

    ### 나이 값 범위 지정하기 ###
    if 18 < age < 29:
        age = 1
    elif 30 < age < 40:
        age = 2
    elif 40 < age < 50:
        age = 3
    elif 50 < age < 60:
        age = 4
    else:
        age = 5

    ### 각 컬럼 범위 array로 묶기 ###
    HD_features = np.array([(BMI, smoking, alchole, stroke, diffwalking, sex, physicalactivity, 
                                sleeptime,  Asthma, kidneydisease, age, diebtic)])
    
    DI_features = np.array([(heartdisease, BMI, smoking, alchole, stroke, diffwalking, sex, physicalactivity, 
                        sleeptime,  Asthma, kidneydisease, age)])
                        
    KD_features = np.array([(heartdisease, BMI, smoking, alchole, stroke, diffwalking, sex, physicalactivity, 
                            sleeptime,  Asthma, age, diebtic)])

    ST_features = np.array([(heartdisease, BMI, smoking, alchole, diffwalking, sex, physicalactivity, 
                            sleeptime,  Asthma, kidneydisease, age, diebtic)])

    AS_features = np.array([(heartdisease, BMI,smoking, alchole, stroke, diffwalking, sex, physicalactivity, 
                            sleeptime, kidneydisease, age, diebtic)])
     

    ### 변수 선언 ###
    HD_count = 0
    DI_count = 0
    KD_count = 0
    ST_count = 0
    AS_count = 0
    All_disease = 0
    another_feature = 0

    ####### 예측모델 결과 #########
    result_HD = HD_ML_predict(HD_model, HD_features)
    if result_HD == 1 or dif_heartdisease == 1:
        HD_count = 1
    else:
        HD_count = 0
    result_DI = DI_ML_predict(DI_model, DI_features)
    if result_DI == 1 or dif_diebtic == 1:
        DI_count = 1
    else:
        DI_count = 0
    result_KD = KD_ML_predict(KD_model, KD_features)
    if result_KD == 1 or dif_kidneydisease == 1:
        KD_count = 1
    else:
        KD_count = 0
    result_ST = ST_ML_predict(ST_model, ST_features)
    if result_ST == 1 or dif_stroke == 1:
        ST_count = 1
    else:
        ST_count = 0
    result_AS = AS_ML_predict(AS_model, AS_features)
    if result_AS == 1 or dif_Asthma == 1:
        AS_count = 1
    else:
        AS_count = 0

    ### 종합 질환에 대한 값 ###
    All_disease = (HD_count + DI_count + KD_count + ST_count + AS_count)
    another_feature = (dif_heartdisease + dif_diebtic + dif_stroke + dif_kidneydisease + dif_Asthma)

    
    ### 머신러닝 예측값 및 질환에 대한 값
    return render_template('result_page.html', HD_data= result_HD, 
                                               DI_data = result_DI, 
                                               KD_data = result_KD,
                                               ST_data = result_ST, 
                                               AS_data = result_AS,
                                               All_disease = All_disease,
                                               another_feature = another_feature,
                                               dif_heartdisease =dif_heartdisease,
                                               dif_diebtic = dif_diebtic,
                                               dif_stroke = dif_stroke,
                                               dif_kidneydisease = dif_kidneydisease,
                                               dif_Asthma = dif_Asthma)

if __name__ =="__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [21/Aug/2022 21:36:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Aug/2022 21:36:22] "GET /static/homepage.css HTTP/1.1" 304 -
127.0.0.1 - - [21/Aug/2022 21:36:22] "GET /static/img/검색아이콘.png HTTP/1.1" 304 -
127.0.0.1 - - [21/Aug/2022 21:36:22] "GET /static/img/it로고.png HTTP/1.1" 304 -
127.0.0.1 - - [21/Aug/2022 21:36:22] "GET /static/img/happyfamily.png HTTP/1.1" 304 -
127.0.0.1 - - [21/Aug/2022 21:36:22] "GET /static/img/shoose.jpg HTTP/1.1" 304 -
127.0.0.1 - - [21/Aug/2022 21:36:22] "GET /static/img/young_woman.jpg HTTP/1.1" 304 -
127.0.0.1 - - [21/Aug/2022 21:36:22] "GET /static/img/family.jpg HTTP/1.1" 304 -
127.0.0.1 - - [21/Aug/2022 21:36:22] "GET /static/img/노부부.jpg HTTP/1.1" 304 -
127.0.0.1 - - [21/Aug/2022 21:36:22] "GET /static/homepage.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Aug/2022 21:36:23] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Aug/2022 21:36:55] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [21/